<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#The-simplest-ageometeorological-crop-model" data-toc-modified-id="The-simplest-ageometeorological-crop-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The simplest ageometeorological crop model</a></span><ul class="toc-item"><li><span><a href="#Main-crop-modelling:-production-efficiency-model-(PEM)" data-toc-modified-id="Main-crop-modelling:-production-efficiency-model-(PEM)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Main crop modelling: production efficiency model (PEM)</a></span></li><li><span><a href="#Crop-limitations" data-toc-modified-id="Crop-limitations-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Crop limitations</a></span><ul class="toc-item"><li><span><a href="#Temperature-control" data-toc-modified-id="Temperature-control-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Temperature control</a></span></li><li><span><a href="#Water-availability" data-toc-modified-id="Water-availability-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Water availability</a></span></li></ul></li></ul></li><li><span><a href="#The-model" data-toc-modified-id="The-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>The model</a></span></li></ul></div>

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import gdal
import pandas as pd
%matplotlib inline

from modis_funcs import *

/home/ucfajlg/.local/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


## The simplest ageometeorological crop model

### Main crop modelling: production efficiency model (PEM)

We can think of the crop turning incoming radiation into carbon, which then gets allocated to different organs (including leave, grain, roots, etc.). The simple PEM model introduced by Monteith is basically that gross primary productivity (GPP) can be written as

$$
GPP(t) = fAPAR\cdot PAR_{inc} \cdot \epsilon,
$$

where $fAPAR$ is the fraction of active photosynthetic radiation absorbed by the canopy (hence related to the amount and colour of leaves), $PAR_{inc}$ is the incoming radiation in the PAR region ($400-700 \textrm{nm}$), and $\epsilon$ is an energy conversion factor (e.g. how many grams of C per square meter do we produce for a given amount of absorbed radiation).

This model is incredibly simple: we do not account for any stressors, pests, management, soils, ... But simplicity has its own advantages!


### Crop limitations

In warm climates, we have two main limitations to crop develoment: water availability and heat.




#### Temperature control

Photosynthesis is controlled by temperature, with typically a "cold" cutoff point, an optimal temperature, and a point where too much heat hampers photosynthetic activity:

$$
f(T)=\begin{cases}
0\; T<T_{thresh,min}\\
\frac{T-T_{thresh,min}}{T_{thresh,opt} - T_{thresh,min}}\; T_{thresh,min} \le T \le T_{thresh,opt}\\
\frac{T_{thresh,max} - T}{T_{thresh,max} - T_{thresh,opt}}\;  T \ge T_{thresh,opt}\\
\end{cases}
$$


In [1]:
def temp_constraint(temp, t_min=12, t_max=41, t_opt=28):
    if temp < t_min:
        return 0.
    elif t_min <= temp <= t_opt:
        return (temp-t_min)/(t_opt - t_min)
    elif temp >= t_opt:
        return max((t_max - temp)/(t_max-t_opt), 0)
temp_axs = np.arange(10, 50, 2)
tt = [temp_constraint(t) for t in temp_axs]
plt.plot(temp_axs, tt)
plt.xlabel("Temperature [deg C]")
plt.ylabel("C assim rate [-]")

NameError: name 'plt' is not defined

#### Water availability
If we consider that vegetation growth is only limited by water availability, a simple model would basically look at water balance: precipitation minus evapotranspiration. If this balance is positive (more rain than ET), we may initially conclude that in principle, crops will not be stressed by drought. One of the simplest ways to calculate ET is to use a version of Hargreaves equation:

$$
ET_{0} = \frac{0.0023}{\lambda}\cdot \left[T_{max} - T_{min} \right]^{0.5}\cdot R_{surf}\left[T_{mean} - 17.8\right]
$$
where the daily reference evapotranspiration $ET_0$ is given in $[mm\cdot d^{1}]$, $T_{max,min,mean}$ are respectively the daily max, min and mean temperatures in degrees C, $R_{surf}$ is the surface radiance in $kJm^{-2}d^{-1}$ and $\lambda$ is the latent heat of vaporisation. We only chose this equation for its simplicity. More accurate and better methods are the FAO56/Penman-Monteith approach although it depends on wind speed, an unreliable quantity in our reanalysis dataset.

A very simple but also effective model for the effect of drought (not considering soil moisture!) is to just define a linear scaling point that relates the precipitation/ET ratio (e.g. available water vs water requirements for unstressed photosynthesis):

$$
f(P)=\begin{cases}
\frac{P}{ET_{0}}\, \textrm{if } 0\le P \le ET_{0}\\
1,\, P>ET_{0}\\
\end{cases}
$$


In [3]:
def water_limitation(precip, et):
    if precip >= et:
        return 1.
    else:
        return precip/et



## The model

We have seen that we can get some data on 